# Memory

In [1]:
import os
from dotenv import load_dotenv
load_dotenv()

True

In [2]:
from langchain_openai.chat_models import ChatOpenAI
from langchain_core.prompts.prompt import PromptTemplate
from langchain_core.prompts.chat import ChatPromptTemplate

# Memory

챗봇으로 하여금 대화(상태)를 '기억'하게끔 한다

Memory maintains Chain state, incorporating context from past runs.


In [ ]:
# Langchain 의 memory 계층
#  BaseMemory --> BaseChatMemory --> <name>Memory  # Examples: ZepMemory, MotorheadMemory

In [3]:
# OpenAI 사에서 제공하는 '기본 API' 도 랭체인 없이 사용 가능.
# 메모리 지원하지 않는다.  이전 대화 기억 못함.  stateless 하다!

# ChatGPT 서비스 는 '메모리' 기능이 탑재되어 있다.
# 챗봇이 이전의 대화 내용과 질문을 기억하고 답할수 있다.


# ConversationBufferMemory

In [4]:
# ConversationBufferMemory
# 대화 내용 '전체'를 저장하는 메모리

# 장점: 단순하다

# 단점:
# => 매번 요청할때마다 '이전 대화 기록 전체' 를 같이 보내야 함.
#  그래야 모델이 전에 일어났던 대화를 보고 이해 할수 있다.
#  대화내용이 길어질수록 메모리도 계속 커지니까 성능적으로도 & 비용적으로도 비효율적이다.


In [5]:
from langchain.memory.buffer import ConversationBufferMemory

In [6]:
memory = ConversationBufferMemory()

memory.save_context(
    {'input': 'Hi!'},    # 유저 입력값
    {'output': 'How are you?'},  # 모델이 유저에게 답한값
)

memory.load_memory_variables({})



C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_19292\3463026985.py:1: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationBufferMemory()


{'history': 'Human: Hi!\nAI: How are you?'}

## return_messages=True
history 에  AIMessage 와 HumanMessage 로 저장된다.

In [7]:
memory = ConversationBufferMemory(return_messages=True)

memory

ConversationBufferMemory(chat_memory=InMemoryChatMessageHistory(messages=[]), return_messages=True)

In [8]:
memory.save_context(
    {'input': 'Hi!'},    # 유저 입력값
    {'output': 'How are you?'},  # 모델이 유저에게 답한값
)

memory.load_memory_variables({})

{'history': [HumanMessage(content='Hi!', additional_kwargs={}, response_metadata={}),
  AIMessage(content='How are you?', additional_kwargs={}, response_metadata={})]}

In [9]:
memory.save_context(
    {'input': 'Hi!'},    # 유저 입력값
    {'output': 'How are you?'},  # 모델이 유저에게 답한값
)

memory.load_memory_variables({})

{'history': [HumanMessage(content='Hi!', additional_kwargs={}, response_metadata={}),
  AIMessage(content='How are you?', additional_kwargs={}, response_metadata={}),
  HumanMessage(content='Hi!', additional_kwargs={}, response_metadata={}),
  AIMessage(content='How are you?', additional_kwargs={}, response_metadata={})]}

In [ ]:
# ↑ 반복하여 save 할수록 memory history 에 Message 들이 쌓인다.

# 대화가 길어질수록 메모리에 내용이 계속 쌓인다 --> 비효율적.

# ConversationBufferWindowMemory

In [10]:
from langchain.memory.buffer_window import ConversationBufferWindowMemory

In [ ]:
# ConversationBufferWindowMemory 는 대화의 '특정 부분만' 을 저장하는 메모리.

# 장점:
#   메모리를 특정 크기로 유지할 수 있다!
#   따라서 모든 대화 내용을 저장하지 않아도 된다!

# 단점:
#   챗봇이 전체 대화가 아닌 '최근 대화' 에만 집중하게 된다.

In [11]:
memory = ConversationBufferWindowMemory(
    return_messages=True,
    k=4,  # 버퍼 윈도우 크기,  몇개의 context(메제지) 를 저장할지 지정
)

C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_19292\1088275114.py:1: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationBufferWindowMemory(


In [12]:
# 도우미 함수, Message 추가하는 함수
def add_message(input, output):
    memory.save_context({"input": input}, {'output': output})

In [13]:
add_message("1", "1")
add_message("2", "2")
add_message("3", "3")
add_message("4", "4")


In [14]:
memory.load_memory_variables({})

{'history': [HumanMessage(content='1', additional_kwargs={}, response_metadata={}),
  AIMessage(content='1', additional_kwargs={}, response_metadata={}),
  HumanMessage(content='2', additional_kwargs={}, response_metadata={}),
  AIMessage(content='2', additional_kwargs={}, response_metadata={}),
  HumanMessage(content='3', additional_kwargs={}, response_metadata={}),
  AIMessage(content='3', additional_kwargs={}, response_metadata={}),
  HumanMessage(content='4', additional_kwargs={}, response_metadata={}),
  AIMessage(content='4', additional_kwargs={}, response_metadata={})]}

In [15]:
add_message("5", "5")

memory.load_memory_variables({})

{'history': [HumanMessage(content='2', additional_kwargs={}, response_metadata={}),
  AIMessage(content='2', additional_kwargs={}, response_metadata={}),
  HumanMessage(content='3', additional_kwargs={}, response_metadata={}),
  AIMessage(content='3', additional_kwargs={}, response_metadata={}),
  HumanMessage(content='4', additional_kwargs={}, response_metadata={}),
  AIMessage(content='4', additional_kwargs={}, response_metadata={}),
  HumanMessage(content='5', additional_kwargs={}, response_metadata={}),
  AIMessage(content='5', additional_kwargs={}, response_metadata={})]}

# ConversationSummaryMemory

In [16]:
# ConversationSummaryMemory 는 LLM 을 사용하여 대화의 요약본 (summary) 생성

In [17]:
llm = ChatOpenAI(temperature=0.1)  # 요약을 위해 모델 호출 필요

In [18]:
from langchain.memory.summary import ConversationSummaryMemory

In [ ]:
# ConversationSummaryMemory
# 메세지를 그대로 저장하는 것이 아니라 Convertaion 의 '요약'을 해준다. LLM 필요

# 장점: 대화의 메세지가 많아질수록 요약을 해주어 토큰의 양도 줄어들어 훨씬 경제적인 방법
# 단점: LLM 호출 발생


In [19]:
memory = ConversationSummaryMemory(llm=llm)  # LLM 사용

C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_19292\1565216213.py:1: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationSummaryMemory(llm=llm)  # LLM 사용


In [20]:
# 도우미 함수
def get_history():
    return memory.load_memory_variables({})

In [21]:
# message 추가
add_message(
    "Hi I'm John, I live in South Korea",    # input
    "Wow that is so cool!"  # output : AI 답변
    )

In [22]:
get_history()

{'history': 'John introduces himself as living in South Korea. The AI responds with enthusiasm, finding it cool.'}

In [ ]:
# ↑ 대화를 '요약' 한 내용으로 기억하고 있다
# 대화의 turn 이 길어질수로 summary 가 각 메세지를 효율적으료 '요약(압축)' 해준다


# ConversationSummaryBufferMemory
- summry + buffer

In [23]:
from langchain.memory.summary_buffer import ConversationSummaryBufferMemory

In [24]:
# ConversationSummaryBufferMemory 는
#   ConversationBufferMemory 와 ConversationSummaryMemory 의 결합형

# 메모리에 보내온 '메세지의 분량'를 지정하여 저장한다.
# 오래된 메세지들 또한 '요약' 하여 저장함.

In [25]:
memory = ConversationSummaryBufferMemory(
    llm=llm,
    max_token_limit=150,  # 최대 가용한 메세지 토큰수 (메세지들이 요약되기전)
    return_messages=True,
)

C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_19292\2618822315.py:1: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationSummaryBufferMemory(


In [26]:
add_message(
    "Hi I'm John, I live in South Korea",    # input
    "Wow that is so cool!"  # output : AI 답변
    )
get_history()

{'history': [HumanMessage(content="Hi I'm John, I live in South Korea", additional_kwargs={}, response_metadata={}),
  AIMessage(content='Wow that is so cool!', additional_kwargs={}, response_metadata={})]}

In [27]:
# 다시 메세지 추가하고 history 확인
add_message(
    "South Korea is so pretty",
    "I wish I could go!!!")
get_history()

{'history': [HumanMessage(content="Hi I'm John, I live in South Korea", additional_kwargs={}, response_metadata={}),
  AIMessage(content='Wow that is so cool!', additional_kwargs={}, response_metadata={}),
  HumanMessage(content='South Korea is so pretty', additional_kwargs={}, response_metadata={}),
  AIMessage(content='I wish I could go!!!', additional_kwargs={}, response_metadata={})]}

In [ ]:
# 아직 요약은 발생하지 않았다.  max_token_limit=150 이하이기 때문.

In [28]:
add_message(
    "How far is Korea from Argentina?",
    "I don't know! Super far!"
)
get_history()

{'history': [HumanMessage(content="Hi I'm John, I live in South Korea", additional_kwargs={}, response_metadata={}),
  AIMessage(content='Wow that is so cool!', additional_kwargs={}, response_metadata={}),
  HumanMessage(content='South Korea is so pretty', additional_kwargs={}, response_metadata={}),
  AIMessage(content='I wish I could go!!!', additional_kwargs={}, response_metadata={}),
  HumanMessage(content='How far is Korea from Argentina?', additional_kwargs={}, response_metadata={}),
  AIMessage(content="I don't know! Super far!", additional_kwargs={}, response_metadata={})]}

In [32]:
add_message(
    "How far is Brazil from Argentina?",
    "I don't know! Super far!"
)
get_history()

{'history': [SystemMessage(content='The human introduces himself as John and mentions that he lives in South Korea.', additional_kwargs={}, response_metadata={}),
  AIMessage(content='Wow that is so cool!', additional_kwargs={}, response_metadata={}),
  HumanMessage(content='South Korea is so pretty', additional_kwargs={}, response_metadata={}),
  AIMessage(content='I wish I could go!!!', additional_kwargs={}, response_metadata={}),
  HumanMessage(content='How far is Korea from Argentina?', additional_kwargs={}, response_metadata={}),
  AIMessage(content="I don't know! Super far!", additional_kwargs={}, response_metadata={}),
  HumanMessage(content='How far is Brazil from Argentina?', additional_kwargs={}, response_metadata={}),
  AIMessage(content="I don't know! Super far!", additional_kwargs={}, response_metadata={}),
  HumanMessage(content='How far is Brazil from Argentina?', additional_kwargs={}, response_metadata={}),
  AIMessage(content="I don't know! Super far!", additional_kwar

In [ ]:
# max_token_limit 에 도달하면, 오래된 메세지들이 요약되고 있을 것을 확인할수 있다. (SystemMessage 확인)

# ★그러나 '요약' 이라는 과정은 API 를 사용한다는 사실을 명심하세요.
# ★'요약' 동작은 비용 지출이 발생되는 부분입니다.

# ConversationKGMemory

In [33]:
# KG : Knowledge Graph

In [34]:
from langchain_community.memory.kg import ConversationKGMemory

In [ ]:
# 대화중에 '엔티티'의 knowledge graph 를 형성한다 => 가장 중요한 것들만 추출한 요약본.
# knowledge graph 는 history 를 가지고 오지 않는다.  대신 '엔티티' 를 가지고 옴


In [35]:
memory = ConversationKGMemory(
    llm=llm,  # 이 또한 LLM 을 사용하는 Memory 다 -> Knowledge Graph 를 만든다. (가장 중요한것만 뽑아낸 요약본)
    return_messages=True,
)

In [36]:
add_message(
    "Hi I'm John, I live in South Korea",    # input
    "Wow that is so cool!"  # output : AI 답변
    )

In [37]:
# 대화의 특정 entity 에 대해 질문할수 있다.
memory.load_memory_variables({'input': 'who is john'})

{'history': [SystemMessage(content='On John: John lives in South Korea.', additional_kwargs={}, response_metadata={})]}

In [38]:
add_message("John likes kimchi", "Wow that is so cool!")

memory.load_memory_variables({"inputs": "What does John like"})


{'history': [SystemMessage(content='On John: John lives in South Korea. John likes kimchi.', additional_kwargs={}, response_metadata={})]}

# 그 밖의 메모리들

In [ ]:
# ConversationTokenBufferMemory 도 있다
#   interaction 의 최대값을 가지고 있는 것 대신에 token의 총 양을 계산하는게 전부다.
#   max_token_limit= 값!  <= ConversationBufferWindowMemory 와 비슷하다 (k= 값)

In [ ]:
# ConversationEntityMemory
#  Entity 를 활용한 메모리도 있다.
#  이는 대화중에 entity 를 추출해 활용

- **참고: Database 와 integration 된 메모리들**

| Memory Class                  | 통합 대상 (Integration)             | 설명                                                           |
| ----------------------------- | ------------------------------- | ------------------------------------------------------------ |
| `RedisChatMessageHistory`     | **Redis**                       | Redis에 메시지 저장. 빠르고 확장 가능한 저장소.                               |
| `SQLChatMessageHistory`       | **SQLite, PostgreSQL 등 SQL DB** | SQL 데이터베이스에 메시지 저장. SQLAlchemy 기반.                           |
| `DynamoDBChatMessageHistory`  | **AWS DynamoDB**                | AWS의 NoSQL DB인 DynamoDB에 대화 저장. 서버리스 환경에서 유용.                |
| `MongoDBChatMessageHistory`   | **MongoDB**                     | 문서 기반 DB인 MongoDB와 통합하여 대화 저장.                               |
| `PostgresChatMessageHistory`  | **PostgreSQL**                  | PostgreSQL 전용 구현 (SQLAlchemy 없이).                            |
| `FileChatMessageHistory`      | **Local 파일**                    | JSON 파일로 로컬에 저장. 간단한 로깅에 적합.                                 |
| `FirestoreChatMessageHistory` | **Google Firestore**            | Firebase 기반 클라우드 NoSQL DB와 통합.                               |
| `ChromaMemory`                | **Chroma (벡터 DB)**              | 벡터 DB에 embedding 형태로 memory 저장. RAG나 유사 검색 기반 memory에 활용 가능. |
| `WeaviateMemory`              | **Weaviate**                    | Weaviate 벡터 DB와 통합된 memory 저장.                               |
| `QdrantMemory`                | **Qdrant**                      | 벡터 기반 memory 저장소로 Qdrant 사용.                                 |


# Memory on LLMChain

In [ ]:
# 1. off-the-shelf chain : LangChain 측에서 '일반적인 목적'을 수행하는 기본제공 chain
# 2. LCEL 사용한 chain : 커스컴 chain

# 위 두가지 형태의 chain 에 각각 메모리를 꽂아 사용해 보자.

In [40]:
# off-the-shelf chain 하나를 사용해보자.
from langchain.chains.llm import LLMChain

In [ ]:
# LLMChain
#  => 'prompt | llm'

In [42]:
llm = ChatOpenAI(temperature=0.1)

memory = ConversationSummaryBufferMemory(
    llm = llm,
    max_token_limit=80,
)

# chain 생성하고 메모리 장착!
chain = LLMChain(
    llm=llm,
    memory=memory,  # 메모리 제공!
    prompt=PromptTemplate.from_template("{question}")
)



C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_19292\3255347543.py:9: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use :meth:`~RunnableSequence, e.g., `prompt | llm`` instead.
  chain = LLMChain(


In [43]:
chain.invoke({'question': "My name is John"})

{'question': 'My name is John',
 'history': '',
 'text': 'Nice to meet you, John! How can I assist you today?'}

In [ ]:
"""
{'question': 'My name is John',
 'history': '',    # <- 최초에는 history 가 비어있다.
 'text': 'Nice to meet you, John! How can I assist you today?'}
"""
None

In [44]:
chain.invoke({'question': "I live in Seoul"})

{'question': 'I live in Seoul',
 'history': 'Human: My name is John\nAI: Nice to meet you, John! How can I assist you today?',
 'text': ", the capital city of South Korea. It is a bustling metropolis with a vibrant culture, delicious food, and a mix of modern skyscrapers and historic palaces. I love exploring the city's neighborhoods, trying new restaurants, and taking in the beautiful views from Namsan Mountain. Seoul is a dynamic city that never fails to impress me with its energy and charm."}

In [ ]:
"""
{'question': 'I live in Seoul',
    ↓ 이전 대화에 대한 history 가 생겼다! <- memory 동작한다는 뜻!
 'history': 'Human: My name is John\nAI: Nice to meet you, John! How can I assist you today?',
 'text': ", the capital city of South Korea. It is a bustling metropolis with a vibrant culture, delicious food, and a mix of modern skyscrapers and historic palaces. I love exploring the city's neighborhoods, trying new restaurants, and taking in the beautiful views from Namsan Mountain. Seoul is a dynamic city that never fails to impress me with its energy and charm."}

"""
None

In [45]:
# 과연 기억하고 있을까?
chain.invoke({'question': "What is my name"})

{'question': 'What is my name',
 'history': "System: The human introduces himself as John and mentions he lives in Seoul, the capital city of South Korea. The AI responds by describing Seoul as a bustling metropolis with a vibrant culture, delicious food, and a mix of modern skyscrapers and historic palaces. The AI expresses its love for exploring the city's neighborhoods, trying new restaurants, and taking in the beautiful views from Namsan Mountain, highlighting Seoul's dynamic energy and charm.",
 'text': "I'm sorry, I do not know your name as I am an AI assistant and do not have access to personal information."}

In [ ]:
"""
{'question': 'What is my name',
 'history': "System: The human introduces himself as John and mentions he lives in Seoul, the capital city of South Korea. The AI responds by describing Seoul as a bustling metropolis with a vibrant culture, delicious food, and a mix of modern skyscrapers and historic palaces. The AI expresses its love for exploring the city's neighborhoods, trying new restaurants, and taking in the beautiful views from Namsan Mountain, highlighting Seoul's dynamic energy and charm.",
 'text': "I'm sorry, I do not know your name as I am an AI assistant and do not have access to personal information."}

어라?  모른다고?

chain 디버깅을 통해 알아보자.

"""
None

## verbose= 

In [47]:
chain = LLMChain(
    llm=llm,
    memory=memory,  # 메모리 제공!
    prompt=PromptTemplate.from_template("{question}"),
    verbose=True,  # chain 을 실행했을때 chain 의 프롬프트 로그들 확인 가능. (디버깅용)
)

chain.invoke(input={'question':"My name is John"})
chain.invoke(input={'question':"I live in Seoul"})
chain.invoke(input={'question':"What is my name?"})







> Entering new LLMChain chain...
Prompt after formatting:
My name is John

> Finished chain.


> Entering new LLMChain chain...
Prompt after formatting:
I live in Seoul

> Finished chain.


> Entering new LLMChain chain...
Prompt after formatting:
What is my name?

> Finished chain.


{'question': 'What is my name?',
 'history': "System: The human, named John, mentions he lives in Seoul, the capital city of South Korea. The AI responds by describing Seoul as a bustling metropolis with a vibrant culture, delicious food, and a mix of modern skyscrapers and historic palaces. The AI expresses its love for exploring the city's neighborhoods, trying new restaurants, and taking in the beautiful views from Namsan Mountain, highlighting Seoul's dynamic energy and charm. John confirms his name and location, prompting the AI to greet him and offer assistance.\nAI: , the capital city of South Korea. It is a bustling metropolis with a vibrant culture, delicious food, and a mix of modern skyscrapers and historic temples. I love exploring the city's neighborhoods, trying new restaurants, and taking in the beautiful views of the Han River. Seoul is a dynamic and exciting place to call home.",
 'text': "I'm sorry, I do not have access to personal information such as your name."}

In [ ]:
"""
> Entering new LLMChain chain...
Prompt after formatting:
My name is John  <----  이게 프롬프트가 가진 전부다

> Finished chain.


> Entering new LLMChain chain...
Prompt after formatting:
I live in Seoul  <----  이게 프롬프트가 가진 전부다

> Finished chain.


> Entering new LLMChain chain...
Prompt after formatting:
What is my name?  <----  이게 프롬프트가 가진 전부다

> Finished chain.


↑ chain 의 prompt 로그 들을 확인할 수 있다.
보다시피 대화의 내역(history) 가 prompt에 계속 추가되진 않는다.

우리가 원하는 어떤 방식으로 prompt에게 대화 기록(history)을 추가해줘야 한다!

↓ 하지만! 메모리는 계속 업데이트 된다.  함 보자!



"""
None

In [48]:
memory.load_memory_variables({})

{'history': "System: John mentions he lives in Seoul, the capital city of South Korea. The AI responds by describing Seoul as a bustling metropolis with a vibrant culture, delicious food, and a mix of modern skyscrapers and historic temples. The AI expresses its love for exploring the city's neighborhoods, trying new restaurants, and taking in the beautiful views of the Han River, highlighting Seoul's dynamic energy and charm. John confirms his name and location, prompting the AI to greet him and offer assistance.\nHuman: What is my name?\nAI: I'm sorry, I do not have access to personal information such as your name."}

In [ ]:
# 위 메모리 내용이 prompt 에 포함되어야 한다!

In [ ]:
# Memory 클래스에게
# 템플릿 안에 콘텐츠를 넣으라고 해주어야 한다.
# memory_key=  에 "chat_history" (예시)

## memory_key=

In [49]:
memory = ConversationSummaryBufferMemory(
    llm=llm,
    max_token_limit=120, 
    memory_key="chat_history",
)

# ↓ template 안에는 memory 가 history 를 저장하도록 한 곳을 적어주기만 하면 된다
# history 까지 담을 괜찮은 템플릿을 준비해보자
template = """
    You are a helpful AI talking to a human.

    {chat_history}
    Human:{question}
    You:
"""

chain = LLMChain(
    llm=llm,
    memory=memory,
    prompt=PromptTemplate.from_template(template),
    verbose=True,
)


In [50]:
chain.invoke({'question': "My name is John"})



> Entering new LLMChain chain...
Prompt after formatting:

    You are a helpful AI talking to a human.

    
    Human:My name is John
    You:


> Finished chain.


{'question': 'My name is John',
 'chat_history': '',
 'text': 'Nice to meet you, John! How can I assist you today?'}

In [ ]:
# {chat_history} 에는 아무 내용도 포맷팅 안됨 <- 메모리 비어 있었다.

In [51]:
chain.invoke({'question': "I live in Seoul"})



> Entering new LLMChain chain...
Prompt after formatting:

    You are a helpful AI talking to a human.

    Human: My name is John
AI: Nice to meet you, John! How can I assist you today?
    Human:I live in Seoul
    You:


> Finished chain.


{'question': 'I live in Seoul',
 'chat_history': 'Human: My name is John\nAI: Nice to meet you, John! How can I assist you today?',
 'text': "That's great to hear! How can I assist you with information or tasks related to Seoul?"}

In [ ]:
# ↑ {chat_history} 에 메모리의 저장된 이전 대화 내용이 포맷팅 되었다.

In [52]:
chain.invoke({'question':'What is my name?'})



> Entering new LLMChain chain...
Prompt after formatting:

    You are a helpful AI talking to a human.

    Human: My name is John
AI: Nice to meet you, John! How can I assist you today?
Human: I live in Seoul
AI: That's great to hear! How can I assist you with information or tasks related to Seoul?
    Human:What is my name?
    You:


> Finished chain.


{'question': 'What is my name?',
 'chat_history': "Human: My name is John\nAI: Nice to meet you, John! How can I assist you today?\nHuman: I live in Seoul\nAI: That's great to hear! How can I assist you with information or tasks related to Seoul?",
 'text': 'Your name is John.'}

In [ ]:
# - 프롬프트 템플릿 안에서 메모리 내용이 들어갈 공간을 준비한다.  (예: chat_history)
# - 메모리를 활용할 템플릿은 원하는대로 작성하면 된다.
# - Memory 클래스에선 history 를 어디에 꽂을지 지정해준다 (memory_key=)


In [54]:
# 과연 요약은 잘 될까?
chain.invoke({'question': "My name is John"})
chain.invoke({'question': "I live in Seoul"})



> Entering new LLMChain chain...
Prompt after formatting:

    You are a helpful AI talking to a human.

    System: The human introduces himself as John. The AI responds by saying, "Nice to meet you, John! How can I assist you today?"
Human: I live in Seoul
AI: That's great to hear! How can I assist you with information or tasks related to Seoul?
Human: What is my name?
AI: Your name is John.
Human: My name is John
AI: Nice to meet you, John! How can I assist you today?
Human: I live in Seoul
AI: That's great to hear! How can I assist you with information or tasks related to Seoul?
    Human:My name is John
    You:


> Finished chain.


> Entering new LLMChain chain...
Prompt after formatting:

    You are a helpful AI talking to a human.

    System: John introduces himself to the AI and mentions that he lives in Seoul. The AI responds by offering assistance with information or tasks related to Seoul.
Human: What is my name?
AI: Your name is John.
Human: My name is John
AI: Nice t

{'question': 'I live in Seoul',
 'chat_history': "System: John introduces himself to the AI and mentions that he lives in Seoul. The AI responds by offering assistance with information or tasks related to Seoul.\nHuman: What is my name?\nAI: Your name is John.\nHuman: My name is John\nAI: Nice to meet you, John! How can I assist you today?\nHuman: I live in Seoul\nAI: That's great to hear! How can I assist you with information or tasks related to Seoul?\nHuman: My name is John\nAI: Nice to meet you, John! How can I assist you today?",
 'text': "That's great to hear! How can I assist you with information or tasks related to Seoul?"}

In [55]:
# 지금까지는 텍스트 기반의 history 였다.

# 과연 Message 기반의 history 는 어떻게 적용하나?

# Chat Based Memory

In [ ]:
# memory 클래스는 memory 를 두가지 방식으로 출력할수 있다는 사실을 기억하자..
# 1. 문자열 형태
# 2. message 형태

In [56]:
memory.load_memory_variables({})

# ↓ 직전의 메모리는 '텍스트 기반'의 메세지 였다.  이 텍스트가 그대로 프롬프트에 포맷팅 된거다.

{'chat_history': "System: John introduces himself to the AI and mentions that he lives in Seoul. The AI responds by offering assistance with information or tasks related to Seoul. When asked, the AI correctly identifies John's name as John.\nHuman: My name is John\nAI: Nice to meet you, John! How can I assist you today?\nHuman: I live in Seoul\nAI: That's great to hear! How can I assist you with information or tasks related to Seoul?\nHuman: My name is John\nAI: Nice to meet you, John! How can I assist you today?\nHuman: I live in Seoul\nAI: That's great to hear! How can I assist you with information or tasks related to Seoul?"}

## return_meesages=True

In [57]:
# 이를 대화 기반의 채팅으로 사용하고 싶다면 변경해야 한다

memory = ConversationSummaryBufferMemory(
    llm=llm,
    max_token_limit=120,
    memory_key="chat_history",
    return_messages=True,   # history 가 문자열기반이 아닌 Message 로 리턴됨.
)


In [ ]:
# 과연 prompt 에는 어떻게 대화 history 들을 넘겨줄수 있을까?
#  단순한 하나의 텍스트가 아니라... Human Message - AI Message - Human Messagbe - AI Message - .... (여러 메세지들)
#  심지여 요약본 발생시 System message 도 있을텐데?

# prompt 에 이를 위한 공간을 어케 만드나?
#  => MessagesPlaceholder

## MessagePlaceHolder

In [60]:
from langchain_core.prompts.chat import MessagesPlaceholder

In [61]:
prompt = ChatPromptTemplate.from_messages([
        ("system", "You are a helpful AI talking to a human"),

        MessagesPlaceholder(variable_name='chat_history'),
        # ↑ variable_name="chat_history"
        #  ConversationSummaryBufferMemory 는
        #    history 에서 message 들을 가져와서 (return_messages=True)
        #    이곳 MessagesPlaceHolder 를 채운다.
        #     AI message, System message, Human message ...
        #     얼마나 많은지 알수 없어도 여기에 채워지는 거다!    
    
        ("human", "{question}"),    
])

In [62]:
chain = LLMChain(
    llm=llm,
    memory=memory,
    prompt=prompt,
    verbose=True,
)

In [63]:
chain.invoke({'question':"My name is John"})



> Entering new LLMChain chain...
Prompt after formatting:
System: You are a helpful AI talking to a human
Human: My name is John

> Finished chain.


{'question': 'My name is John',
 'chat_history': [HumanMessage(content='My name is John', additional_kwargs={}, response_metadata={}),
  AIMessage(content='Hello John! How can I assist you today?', additional_kwargs={}, response_metadata={})],
 'text': 'Hello John! How can I assist you today?'}

In [ ]:
"""
> Entering new LLMChain chain...
Prompt after formatting:
System: You are a helpful AI talking to a human   <-- 'system' message
                                                <-- MessagesPlaceholder 에는 아직 history 없다.
Human: My name is John                         <-- 'human' message

> Finished chain.
"""
None

In [64]:
chain.invoke({'question':"I live in Seoul"})



> Entering new LLMChain chain...
Prompt after formatting:
System: You are a helpful AI talking to a human
Human: My name is John
AI: Hello John! How can I assist you today?
Human: I live in Seoul

> Finished chain.


{'question': 'I live in Seoul',
 'chat_history': [HumanMessage(content='My name is John', additional_kwargs={}, response_metadata={}),
  AIMessage(content='Hello John! How can I assist you today?', additional_kwargs={}, response_metadata={}),
  HumanMessage(content='I live in Seoul', additional_kwargs={}, response_metadata={}),
  AIMessage(content='Seoul is a vibrant city with a rich history and culture. Is there anything specific you would like to know or discuss about Seoul?', additional_kwargs={}, response_metadata={})],
 'text': 'Seoul is a vibrant city with a rich history and culture. Is there anything specific you would like to know or discuss about Seoul?'}

In [ ]:
"""
> Entering new LLMChain chain...
Prompt after formatting:
System: You are a helpful AI talking to a human  <--- ★
Human: My name is John                           <--- ★ MessagesPlaceHolder 의 history
AI: Hello John! How can I assist you today?      <--- ★  MessagesPlaceHolder 의 history
Human: I live in Seoul                           <--- ★

> Finished chain.

"""

In [65]:
chain.invoke({'question':"What is my name?"})



> Entering new LLMChain chain...
Prompt after formatting:
System: You are a helpful AI talking to a human
Human: My name is John
AI: Hello John! How can I assist you today?
Human: I live in Seoul
AI: Seoul is a vibrant city with a rich history and culture. Is there anything specific you would like to know or discuss about Seoul?
Human: What is my name?

> Finished chain.


{'question': 'What is my name?',
 'chat_history': [HumanMessage(content='My name is John', additional_kwargs={}, response_metadata={}),
  AIMessage(content='Hello John! How can I assist you today?', additional_kwargs={}, response_metadata={}),
  HumanMessage(content='I live in Seoul', additional_kwargs={}, response_metadata={}),
  AIMessage(content='Seoul is a vibrant city with a rich history and culture. Is there anything specific you would like to know or discuss about Seoul?', additional_kwargs={}, response_metadata={}),
  HumanMessage(content='What is my name?', additional_kwargs={}, response_metadata={}),
  AIMessage(content='Your name is John.', additional_kwargs={}, response_metadata={})],
 'text': 'Your name is John.'}

# LCEL Based Memory

In [ ]:
# 수동으로 만들어진 chain  에
# langchain expression language(LCEL) 를 활용해서 memory 를 추가하는 방법을 배워보자

In [66]:
llm = ChatOpenAI(temperature=0.1)

memory = ConversationSummaryBufferMemory(
    llm=llm,
    max_token_limit=120,
    memory_key="chat_history",
    return_messages=True,
)

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a helpful AI talking to a human"),
        MessagesPlaceholder(variable_name="chat_history"),
        ("human", "{question}"),
    ]
)

In [69]:
chain = prompt | llm

chain.invoke({
    "chat_history": memory.load_memory_variables({})['chat_history'],
    "question": 'My name is John'
})

AIMessage(content='Hello John! How can I assist you today?', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 10, 'prompt_tokens': 24, 'total_tokens': 34, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'id': 'chatcmpl-CjOqv1O0WENszJFY0XkDZDP1cnJEL', 'finish_reason': 'stop', 'logprobs': None}, id='run--ccfc6cbc-b58b-4fce-8a22-d6434eda9f4b-0', usage_metadata={'input_tokens': 24, 'output_tokens': 10, 'total_tokens': 34, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

In [ ]:
# ↑ 위 방법도 가능은 하다...
# 단, 위 접근 방식의 문제는
# 우리가 chain 을 호출할 때마다 chat_history 도 추가해줘야 한다는 거다.

# ↓ 이보다 더 좋은 방법도 있다. 바로 'Runnables' 라는 것을 사용하는 것이다

## RunnablePassthrough

In [70]:
from langchain_core.runnables.passthrough import RunnablePassthrough

In [74]:
# 메모리 변수 값 리턴하는 함수
def load_memory():
    return memory.load_memory_variables({})['chat_history']

chain = RunnablePassthrough.assign(chat_history=load_memory) | prompt | llm

chain.invoke({"question": "My name is John"})

"""
↑ 위를 실행하게 되면 가장 먼저 load_memory() 를 호출한다.
prompt가 필요로 하는 chat_history= 키 내부에 넣는다.

이는 마치
chain.invoke({
    "chat_history": load_memory(),  <- 요렇게 한것과 동일하다.
    "question": "My name is John",
})

prompt 가 포맷팅 되기 전에 load_memory() 함수가 실행되는 것을 허락해준다.


↓ 그런데 실행하면 에러 날거다!

TypeError: load_memory() takes 0 positional arguments but 1 was given

호출되는 함수 (load_memory) 에 한개의 input 값이 전달되기 때문에 그렇다.

↓ 도대체 어떤값이 load_memory() 에 전달되는지 확인해보자!
"""
None




TypeError: load_memory() takes 0 positional arguments but 1 was given

In [75]:
# 도대체 어떤값이 load_memory() 에 전달되는지 확인해보자!

llm = ChatOpenAI(temperature=0.1)

memory = ConversationSummaryBufferMemory(
    llm=llm,
    max_token_limit=120,
    memory_key="chat_history",
    return_messages=True,
)

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a helpful AI talking to a human"),
        MessagesPlaceholder(variable_name="chat_history"),
        ("human", "{question}"),
    ]
)

def load_memory(input):
    print("😎load_memory():", input)  # 확인!
    return memory.load_memory_variables({})["chat_history"]

chain = RunnablePassthrough.assign(chat_history=load_memory) | prompt | llm

chain.invoke({
    "question": "My name is John",
})


😎load_memory(): {'question': 'My name is John'}


AIMessage(content='Hello John! How can I assist you today?', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 10, 'prompt_tokens': 24, 'total_tokens': 34, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'id': 'chatcmpl-CjP1gYV2DmIX4BqhB6c4mV7KV0WuV', 'finish_reason': 'stop', 'logprobs': None}, id='run--8515dde3-0b35-44e1-aa12-b6249efb6ff5-0', usage_metadata={'input_tokens': 24, 'output_tokens': 10, 'total_tokens': 34, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

In [ ]:
"""
load_memory() {'question': 'My name is John'}

체인에 있는 모든 컴포넌트는 input 을 받을거고, 또 output 을 줄거다.
이게 랭체인의 핵심이다.  모든 것이 input을 얻을거고, 그 후엔 output 을 줄거다.


chain.invoke(
  {     <--  이 dict 는 chain 의 첫번째 아이템의 input 이 되는거다.
             바로 그게 load_memory() 의 input 이 된것이다!   규칙이다!
    "question": "My name is John",
  }
)

이후 load_memory() 를 실행히킨 결과로 얻은것이 chat_history 속성으로 들어가
체인의 다음 요소 로 전달되는 것이다.

RunnablePassThrough 는 최초 전달받은 'question' 을 체인의 다음요소 까지 전달 (key 값 변경 없이!)

"""
None



In [76]:
# chain 호출 결과를 다시 메모리에 저장하려면?

In [77]:
llm = ChatOpenAI(temperature=0.1)

memory = ConversationSummaryBufferMemory(
    llm=llm,
    max_token_limit=120,
    memory_key="chat_history",
    return_messages=True,
)

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a helpful AI talking to a human"),
        MessagesPlaceholder(variable_name="chat_history"),
        ("human", "{question}"),
    ]
)

def load_memory(input):
    print("😎load_memory():", input)  # 확인!
    return memory.load_memory_variables({})["chat_history"]

chain = RunnablePassthrough.assign(chat_history=load_memory) | prompt | llm

# 체인 호출 함수를 만들어 보자
def invoke_chain(question):
    result = chain.invoke({
        "question": question,
    })
    # 체인 호출 결과를 메모리에 저장
    memory.save_context(
        {"input": question},
        {"output": result.content},
    )
    print(result)

In [78]:
invoke_chain("My name is John")

😎load_memory(): {'question': 'My name is John'}
content='Hello John! How can I assist you today?' additional_kwargs={'refusal': None} response_metadata={'token_usage': {'completion_tokens': 10, 'prompt_tokens': 24, 'total_tokens': 34, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'id': 'chatcmpl-CjP7bFKS7eI6XCEUHUYiiZJXwO2YI', 'finish_reason': 'stop', 'logprobs': None} id='run--8998807b-edaa-4c8a-befb-1effd0bdd5d8-0' usage_metadata={'input_tokens': 24, 'output_tokens': 10, 'total_tokens': 34, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}}


In [79]:
invoke_chain("What is my name?")

😎load_memory(): {'question': 'What is my name?'}
content='Your name is John.' additional_kwargs={'refusal': None} response_metadata={'token_usage': {'completion_tokens': 5, 'prompt_tokens': 47, 'total_tokens': 52, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'id': 'chatcmpl-CjP8ERxLFgDkIREYB9BCj2dLFihhp', 'finish_reason': 'stop', 'logprobs': None} id='run--8c4b79e1-9943-4bca-a5d4-55ba1035e56c-0' usage_metadata={'input_tokens': 47, 'output_tokens': 5, 'total_tokens': 52, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}}


In [80]:
get_history()

{'chat_history': [HumanMessage(content='My name is John', additional_kwargs={}, response_metadata={}),
  AIMessage(content='Hello John! How can I assist you today?', additional_kwargs={}, response_metadata={}),
  HumanMessage(content='What is my name?', additional_kwargs={}, response_metadata={}),
  AIMessage(content='Your name is John.', additional_kwargs={}, response_metadata={})]}

## memory_key= 삭제

In [81]:
# memory_key= 의 기본값은 'history' 였다.

In [85]:
llm = ChatOpenAI(temperature=0.1)

memory = ConversationSummaryBufferMemory(
    llm=llm,
    max_token_limit=120,
    # memory_key="chat_history",  # 삭제!
    return_messages=True,
)

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a helpful AI talking to a human"),
        MessagesPlaceholder(variable_name="history"),  # 변경!
        ("human", "{question}"),
    ]
)

def load_memory(input):
    print("😎load_memory():", input)  
    return memory.load_memory_variables({})["history"]  # 변경!

                                  # ↓ 변경!
chain = RunnablePassthrough.assign(history=load_memory) | prompt | llm

def invoke_chain(question):
    result = chain.invoke({
        "question": question,
    })
    memory.save_context(
        {"input": question},
        {"output": result.content},
    )
    print(result)

In [86]:
invoke_chain("My name is John")

😎load_memory(): {'question': 'My name is John'}
content='Hello John! How can I assist you today?' additional_kwargs={'refusal': None} response_metadata={'token_usage': {'completion_tokens': 10, 'prompt_tokens': 24, 'total_tokens': 34, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'id': 'chatcmpl-CjPEO55j6ZFmbPlm2X0ffnJxupp0O', 'finish_reason': 'stop', 'logprobs': None} id='run--22cbc1f5-f153-4705-8d19-8001cb4ecdfb-0' usage_metadata={'input_tokens': 24, 'output_tokens': 10, 'total_tokens': 34, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}}


In [87]:
invoke_chain("What is my name?")

😎load_memory(): {'question': 'What is my name?'}
content='Your name is John.' additional_kwargs={'refusal': None} response_metadata={'token_usage': {'completion_tokens': 5, 'prompt_tokens': 47, 'total_tokens': 52, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'id': 'chatcmpl-CjPEhdhsffc4TSH5FBrmwqurWVV8T', 'finish_reason': 'stop', 'logprobs': None} id='run--3bd25d90-d6da-4af9-8a22-2dd9dd2161ee-0' usage_metadata={'input_tokens': 47, 'output_tokens': 5, 'total_tokens': 52, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}}


In [88]:
llm = ChatOpenAI(temperature=0.1)

memory = ConversationSummaryBufferMemory(
    llm=llm,
    max_token_limit=120,
    # memory_key="chat_history",  # 삭제!
    return_messages=True,
)

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a helpful AI talking to a human"),
        MessagesPlaceholder(variable_name="history"),  # 변경!
        ("human", "{question}"),
    ]
)

def load_memory():
    return memory.load_memory_variables({})["history"]  # 변경!
        
chain = prompt | llm

def invoke_chain(question):
    result = chain.invoke({
        "history": load_memory(),
        "question": question,
    })
    memory.save_context(
        {"input": question},
        {"output": result.content},
    )
    print(result)

invoke_chain("My name is John")
invoke_chain("What is my name?")    

content='Hello John! How can I assist you today?' additional_kwargs={'refusal': None} response_metadata={'token_usage': {'completion_tokens': 10, 'prompt_tokens': 24, 'total_tokens': 34, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'id': 'chatcmpl-CjPJBpNM9NgV75KNIIj8LV8gipcgX', 'finish_reason': 'stop', 'logprobs': None} id='run--91ea8bc0-ff0f-4678-aa4b-f7adade5838d-0' usage_metadata={'input_tokens': 24, 'output_tokens': 10, 'total_tokens': 34, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}}
content='Your name is John.' additional_kwargs={'refusal': None} response_metadata={'token_usage': {'completion_tokens': 5, 'prompt_tokens': 47, 'total_tokens': 52, 'completion_tokens_details': {'accepted_prediction_to